## Introduction à la quantization 

Laurent cetinsoy

Les réseaux de neurones prennent beaucoup de place et il peut être difficile de les faire rentrer sur certains dispositifs embarqués. 

Il existe plusieurs méthodes pour réduire la taille et augmenter la vitesse d'executer des réseaux de neurone. Par exemple il y a ce qu'on appelle la quantization et le pruning.

Dans ce notebook on va faire une introduction à la quantization avec la librairie tensorflow lite.


## Quantization post training

Dans un premier temps on va quantifier notre réseau après l'avoir entraîné normalement. 


Entraîner un réseau de neurone convolutionnel simple avec keras pour faire de la classification MNIST (ou un autre dataset simple de votre choix si (vous en avez marre de ce dataset - https://keras.io/api/datasets/)




In [ ]:
from tensorflow.keras.datasets.mnist import load_data
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D
import matplotlib.pyplot as plt

dataset = load_data()
train, test = dataset
X_train, y_train = train
X_test, y_test = test

print(type(X_train))
# (X_train.shape) Pour afficher les dimensions

X_train = X_train.reshape(-1, 28, 28, 1) / 255 # -1 signifie qu il calcule auto le nb d elements
#X_train = X_train.reshape(60000, 28, 28, 1)
# /255 signifie une normalisation des donnees

# En augmentant le nombre de couches ou les params, le modele ne se televerse plus sur l arduino
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=(2, 2), activation="relu", input_shape=(28, 28, 1)))
model.add(MaxPooling2D((2, 2), strides=None))
model.add(Flatten())
model.add(Dense(units=10, activation='softmax'))

# Modele vise a l origine mais qu on a pas pu implementer par des contraintes de poids :
#model.add(Conv2D(filters=32, kernel_size=(3, 3), activation="relu", input_shape=(28, 28, 1)))
#model.add(Conv2D(filters=64, kernel_size=(3, 3), activation="relu", input_shape=(28, 28, 1)))
#model.add(MaxPooling2D((2, 2), strides=None))
#model.add(Flatten())
#model.add(Dense(units=10, activation='softmax', input_shape=(28**2, )))

model.compile(loss='sparse_categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

model.fit(X_train, y_train, validation_data=(X_test, y_test))
# Validation_data pour calculer un score sur le test
# On peut rajouter des epochs

<class 'numpy.ndarray'>
1875/1875 [==============================] - 27s 14ms/step - loss: 0.3175 - accuracy: 0.9096 - val_loss: 155.8039 - val_accuracy: 0.7285


Afficher le nombre de paramètre du modèle

In [ ]:
model.summary() # DOnne la structure du reseau et le nb de params par couche
# Pooling pour reduire la taille de l image
# Le nombre de paramatres va drastiquement diminuer avec unn max pooling

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 27, 27, 16)        80        
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 13, 13, 16)       0         
 2D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 2704)              0         
                                                                 
 dense_2 (Dense)             (None, 10)                27050     
                                                                 
Total params: 27,130
Trainable params: 27,130
Non-trainable params: 0
_________________________________________________________________


Sauvegarder votre modèle et afficher la taille du fichier. Si on applique une bête règle de trois, quelle est la taille occupée par paramètre ? 

In [ ]:
import joblib
import os.path

joblib.dump(model, "embedded_network.joblib")

file_size = os.path.getsize("embedded_network.joblib")
print(f"\nLe fichier a une taille de : {file_size} octets.")

# Pour obtenir la taille d'un parametre, on va diviser la taille du fichier par le nombre de parametres.
param_size = file_size / 387466
print(f"La taille d'un parametre est de : {param_size} octets")

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers
......conv2d
.........vars
............0
............1
......dense
.........vars
............0
............1
......flatten
.........vars
......max_pooling2d
.........vars
...metrics
......mean
.........vars
............0
............1
......mean_metric_wrapper
.........vars
............0
............1
...optimizer
......vars
.........0
.........1
.........2
.........3
.........4
.........5
.........6
.........7
.........8
...vars
Keras model archive saving:
File Name                                             Modified             Size
metadata.json                                  2023-03-27 19:48:39           64
config.json                                    2023-03-27 19:48:39         1910
variables.h5                                   2023-03-27 19:48:39       350072

Le fichier a une taille de : 352463 octets.
La taille d'un parametre est de : 0.9096617509665365 octets


On va maintenant convertir notre modèle keras en modèle tensorflow lite. 

Installer la librairie tensorflow lite créer une instance de la class TFLiteConverter à partir de votre modèle keras


In [ ]:
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_keras_model(model)

Convertir votre modèle et le sauvegarder dans un fichier nommé model.tflite. Sa taille est-elle plus petite ? 

In [ ]:
tflite_model = converter.convert()

joblib.dump(tflite_model, "model.tflite")

tflite_size = os.path.getsize("model.tflite")
print(f"\nCe nouveau fichier model.tflite fait {tflite_size} octets.")
print("Il est donc plus petit que l ancien fichier.")


Ce nouveau fichier model.tflite fait 110897 octets.
Il est donc plus petit que l ancien fichier.


On va maintenant spécifier des optimisations au converter. 

1. Recréer un converter

2. modifier son attribut optimizations pour ajouter une liste d'optimisation avec la valeur tf.lite.Optimize.DEFAULT

3. Relancer la conversion du modèle, sauvegarder le modèle et regarder la taille du fichier généré

In [ ]:
#1
new_converter = tf.lite.TFLiteConverter.from_keras_model(model)

#2
new_converter.optimizations = [tf.lite.Optimize.DEFAULT]

#3
new_tflite_model = new_converter.convert()
joblib.dump(new_tflite_model, "new_model.tflite")

new_tflite_size = os.path.getsize("new_model.tflite")
print(f"\nApres optimisation, ce nouveau fichier model.tflite fait {new_tflite_size} octets.")
print("Il est donc plus petit que l ancien fichier.")


Apres optimisation, ce nouveau fichier model.tflite fait 29874 octets.
Il est donc plus petit que l ancien fichier.


Quelle type  de quantization Optimize.Default, utilise-t-elle ?


L'option Optimize.DEFAULT(tf.lite.Optimize.DEFAULT)utilisée par le convertisseur TFLite applique ici une optimisation complète du modèle, y compris la quantification dynamique des poids, l'optimisation des formats de données d'entrée et de sortie et la fusion arithmétique. Cette option réduit également la taille du modèle en utilisant des techniques de compression tout en maintenant une qualité de performance comparable.


## Quantization aware training 

Dans cette section on va s'intéresser à l'entraînement sensible à la quantification. L'idée est de simuler les effets de la quantification pendant l'entraînement pour que le modèle ajuste les poids afin de tenir ocmpte de la quantification. 

Reprendre le modèle entraîné sur MNIST


In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 27, 27, 16)        80        
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 13, 13, 16)       0         
 2D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 2704)              0         
                                                                 
 dense_2 (Dense)             (None, 10)                27050     
                                                                 
Total params: 27,130
Trainable params: 27,130
Non-trainable params: 0
_________________________________________________________________


A l'aide de la fonction quantize de tensorflow_model_optimization, créer une seconde version de votre modèle entraîné nommé qat_model

In [ ]:
!pip install tensorflow-model-optimization
import tensorflow_model_optimization as tfmot

qat_model = tfmot.quantization.keras.quantize_model(model)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Compiler le modèle

In [ ]:
qat_model.compile(loss='sparse_categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

Afficher le summury du modèle. D'après vous ce modèle est-il quantifié ? 

In [ ]:
qat_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 quantize_layer_2 (QuantizeL  (None, 28, 28, 1)        3         
 ayer)                                                           
                                                                 
 quant_conv2d_2 (QuantizeWra  (None, 27, 27, 16)       115       
 pperV2)                                                         
                                                                 
 quant_max_pooling2d_1 (Quan  (None, 13, 13, 16)       1         
 tizeWrapperV2)                                                  
                                                                 
 quant_flatten_2 (QuantizeWr  (None, 2704)             1         
 apperV2)                                                        
                                                                 
 quant_dense_2 (QuantizeWrap  (None, 10)              

Le modèle est quantifié car après l'utilisation de la fonction tfmot.quantization.keras.quantize_model(), le nom des couches est précédé par le prefixe 'quant' qui signifie que le modèle est quantifié

Réentraîner votre modèle sur un sous ensemble des modèles sur une ou deux epochs et afficher la performance sur le train et test set

In [ ]:
qat_model.fit(X_train, y_train, epochs=1)

train_loss, train_acc = qat_model.evaluate(X_train, y_train)
print("Train loss:", train_loss)
print("Train accuracy:", train_acc)

test_loss, test_acc = qat_model.evaluate(X_test, y_test)
print("Test loss:", test_loss)
print("Test accuracy:", test_acc)

1875/1875 [==============================] - 14s 7ms/step - loss: 0.1136 - accuracy: 0.9690
Train loss: 0.11355474591255188
Train accuracy: 0.968999981880188
313/313 [==============================] - 2s 7ms/step - loss: 0.2181 - accuracy: 0.9364
Test loss: 0.21807856857776642
Test accuracy: 0.9363999962806702


Convertir votre modèle avec TFLite

In [ ]:
qat_converter = tf.lite.TFLiteConverter.from_keras_model(qat_model)
qat_tflite_model = qat_converter.convert()

In [ ]:
import numpy as np

interpreter = tf.lite.Interpreter(model_content=new_tflite_model)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

accuracy = 0
count = 0
input_details[0]['shape']

for i in range(len(y_test)):
    input_data = X_test[i].reshape(1, 28, 28,1).astype('float32')
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()
    output = interpreter.get_tensor(output_details[0]['index'])
    #print(output, )
    #print(y_test[i], np.argmax(output))

    if np.argmax(output) == y_test[i]:
      count += 1

In [ ]:
# On calcul l'accuracy 

accuracy = count / (i+1)
print(count, i+1)
print(accuracy)

In [ ]:
# On fait pareil avec QAT

qat_tflite_model = qat_converter.convert()

qat_interpreter = tf.lite.Interpreter(model_content=qat_tflite_model)
qat_interpreter.allocate_tensors()

qat_input_details = qat_interpreter.get_input_details()
qat_output_details = qat_interpreter.get_output_details()

qat_accuracy = 0
qat_count = 0
qat_input_details[0]['shape']

for i in range(len(y_test)):
    qat_input_data = X_test[i].reshape(1, 28, 28,1).astype('float32')
    qat_interpreter.set_tensor(qat_input_details[0]['index'], qat_input_data)
    qat_interpreter.invoke()
    qat_output = qat_interpreter.get_tensor(qat_output_details[0]['index'])

    if np.argmax(qat_output) == y_test[i]:
      qat_count += 1

In [ ]:
# On calcul l'accuracy pour QAT

qat_accuracy = qat_count / (i+1)
print(qat_count, i+1)
print(qat_accuracy)

Comparer la performance du modèle Quantified aware training, au modèle original et au modèle quantifié post training

In [ ]:
# On compare les performances sur le jeu de donnees de test
# Evaluation du modele original
original_test_loss, original_test_acc = model.evaluate(X_test, y_test)

# Evaluation du modele quantifie post training


# Evaluation du modele Quantified aware training
qat_test_loss, qat_test_acc = qat_model.evaluate(X_test, y_test)

# Comparaison
print("Test Loss : Original : ", original_test_loss, " | Post Training : , pt_test_loss,  | Aware Training : ", qat_test_loss)
print("Test Accuracy : Original :", original_test_acc, " | Post Training : , pt_test_acc,  | Aware Training : ", qat_test_acc)

Sauvegarder le modèle QAT et comparer les tailles des modèles

In [ ]:
joblib.dump(qat_tflite_model, "qat_model.tflite")

qat_tflite_size = os.path.getsize("qat_model.tflite")
print(f"\nCe nouveau fichier qat_model.tflite fait {qat_tflite_size} octets.")
print("Il est donc plus petit que l ancien fichier.")

In [ ]:
!xxd -i qat_model.tflite > qat_model_data.cc

In [ ]:
!xxd -i qat_model.tflite > qat_model_data.h

Bonus : déployer votre modèle sur votre téléphone ou un dispositif embarqué si vous en disposez d'un. 

In [ ]:
!more qat_model_data.cc

unsigned char qat_model_tflite[] = {
  0x80, 0x04, 0x42, 0xb0, 0xc3, 0x06, 0x00, 0x1c, 0x00, 0x00, 0x00, 0x54,
  0x46, 0x4c, 0x33, 0x14, 0x00, 0x20, 0x00, 0x1c, 0x00, 0x18, 0x00, 0x14,
  0x00, 0x10, 0x00, 0x0c, 0x00, 0x00, 0x00, 0x08, 0x00, 0x04, 0x00, 0x14,
  0x00, 0x00, 0x00, 0x1c, 0x00, 0x00, 0x00, 0x90, 0x00, 0x00, 0x00, 0xe8,
  0x00, 0x00, 0x00, 0x4c, 0xa6, 0x06, 0x00, 0x5c, 0xa6, 0x06, 0x00, 0xec,
  0xc2, 0x06, 0x00, 0x03, 0x00, 0x00, 0x00, 0x01, 0x00, 0x00, 0x00, 0x04,
  0x00, 0x00, 0x00, 0x16, 0x56, 0xf9, 0xff, 0x0c, 0x00, 0x00, 0x00, 0x1c,
  0x00, 0x00, 0x00, 0x40, 0x00, 0x00, 0x00, 0x0f, 0x00, 0x00, 0x00, 0x73,
  0x65, 0x72, 0x76, 0x69, 0x6e, 0x67, 0x5f, 0x64, 0x65, 0x66, 0x61, 0x75,
  0x6c, 0x74, 0x00, 0x01, 0x00, 0x00, 0x00, 0x04, 0x00, 0x00, 0x00, 0x8c,
  0xff, 0xff, 0xff, 0x1d, 0x00, 0x00, 0x00, 0x04, 0x00, 0x00, 0x00, 0x0d,
  0x00, 0x00, 0x00, 0x71, 0x75, 0x61, 0x6e, 0x74, 0x5f, 0x64, 0x65, 0x6e,
  0x73, 0x65, 0x5f, 0x31, 0x00, 0x00, 0x00, 0x01, 0x00, 0x00, 0x00, 0x04,
 

Bonus : Obtenir un modèle qui sera à la fois quantifié et élagué (prunned) en s'aidant de la documentation (https://www.tensorflow.org/model_optimization/guide/pruning/pruning_with_keras)

A l'aide de tensorflow lite / tensorflow lite micro 

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=0d51e245-899d-41d6-b23b-cf3e4bbbc6ea' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>